# Run This Cell to get the latest course materials (IMPORTANT-BACKUP YOUR CURRENT FILES FIRST)

In [ ]:
!git pull

# AIThaiGen SmartFarm Data Collection
<img src="AIThaiGenHW.png" />

# Open Dashboard
- Run the cell below
- Type E-Mail and Password
- Click "LOG IN" button
- Select Device

In [ ]:
import aithaigen_hwdashboard
dashboard = aithaigen_hwdashboard.HWDashBoardWidget()
dashboard

# Save data to csv file

In [ ]:
import pandas as pd

## For example, if you name the data as collection-1 to 4
data1 = pd.DataFrame.from_dict(dashboard.data['collection-1'])
data2 = pd.DataFrame.from_dict(dashboard.data['collection-2'])
data3 = pd.DataFrame.from_dict(dashboard.data['collection-3'])
data4 = pd.DataFrame.from_dict(dashboard.data['collection-4'])
data = pd.concat([data1, data2, data3, data4])

pd.DataFrame.from_dict(data).to_csv('farm.csv')

In [ ]:
data

# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score        # Accuracy metrics


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical

# Prepare Input Data

In [ ]:
data = pd.read_csv("farm.csv").drop(["timeStamp", "Unnamed: 0"], axis=1)  # read csv file

X = data.drop('class', axis=1) # collect only input data

y_num, conditions = pd.factorize(data['class'], sort=True) # Change classes to numerical data

y = to_categorical(y_num).astype(int) # one of N representation

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234) # Split data into train and test

In [ ]:
X_train

# Train Model
<img src="NN2.jpg"/>

In [ ]:
model = Sequential()
model.add(Dense(5, activation='relu'))                      # Hidden layer
model.add(Dense(actions.shape[0], activation='softmax'))    # Output layer

model.compile(optimizer='Adam', loss='categorical_crossentropy', 
              metrics=['categorical_accuracy'])

model.fit(X_train, y_train, epochs=200)

model.summary()

# Test Model
<img src="confusion_matrix.jpg"/>

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

# ======================== Test Model ========================================
y_prob = model.predict(X_test)
# =============================================================================


ytrue = np.argmax(y_test, axis=1).tolist() # target class
yhat = np.argmax(y_prob, axis=1).tolist()  # output class

cfm = multilabel_confusion_matrix(ytrue, yhat)


ConfusionMatrixDisplay.from_predictions(ytrue, yhat)

In [ ]:
accuracy_score(ytrue, yhat)

# Save Model

In [ ]:
model.save('action') 

# Load Model

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('action')

# Use Model in Realtime

In [ ]:
# Represents light intensity values
pump_actions = np.array([0.5, 0.0, 1.0])

test_data = pd.DataFrame.from_dict([dashboard.latest_smartfarm_data]).drop('timeStamp', axis=1)

y = model.predict(test_data)

print(conditions[np.argmax(y)])

dashboard.set_light_intensity(actions[np.argmax(y)])